In [20]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Dense
from keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks

In [15]:
train_images_array = np.load("train_images.npy")
train_labels_array = np.load("train_labels.npy")

In [16]:
train_images_array.shape

(7964, 224, 224, 3)

In [17]:
train_labels_array = to_categorical(train_labels_array, dtype="int")
train_labels_array

array([[0, 1],
       [0, 1],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]])

In [4]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

2022-06-07 19:43:09.673969: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
for layers in (vggmodel.layers)[:15]:
    layers.trainable = False

In [6]:
vggmodel.layers[-2].output

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'fc2')>

In [7]:
dense_layer2 = vggmodel.layers[-2].output
prediction_layer = Dense(units=2, activation="softmax")(dense_layer2)

vgecocup = Model(inputs=vggmodel.input, outputs=prediction_layer)
optimizer = Adam(learning_rate=0.0001)
vgecocup.compile(optimizer=optimizer,
                 loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
vgecocup.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
X_train_model, X_test_model, y_train_model, y_test_model = train_test_split(
    train_images_array, train_labels_array, test_size=0.10)

In [23]:
early_stopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1, restore_best_weights=True) 
hist = vgecocup.fit(
    x=X_train_model,
    y=y_train_model,
    verbose=1,
    batch_size=10,
    steps_per_epoch=10,
    epochs=50,
    validation_data=(X_test_model, y_test_model),
    callbacks=early_stopping
)

Epoch 1/50
10/10 [==============================] - 217s 24s/step - loss: 0.0707 - accuracy: 0.9800 - val_loss: 0.0601 - val_accuracy: 0.9749
Epoch 2/50
10/10 [==============================] - 231s 25s/step - loss: 0.0438 - accuracy: 0.9900 - val_loss: 0.0742 - val_accuracy: 0.9824
Epoch 3/50
10/10 [==============================] - 221s 24s/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.1516 - val_accuracy: 0.9749
Epoch 4/50
10/10 [==============================] - 216s 24s/step - loss: 0.3125 - accuracy: 0.9600 - val_loss: 0.0876 - val_accuracy: 0.9724
Epoch 5/50
10/10 [==============================] - 217s 24s/step - loss: 0.1506 - accuracy: 0.9800 - val_loss: 0.1147 - val_accuracy: 0.9686
Epoch 6/50
10/10 [==============================] - 221s 24s/step - loss: 0.1143 - accuracy: 0.9600 - val_loss: 0.0751 - val_accuracy: 0.9711
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping


In [24]:
hist.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [27]:
vgecocup.save("models/vgecocup")

2022-06-07 21:58:17.754916: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: vgecocup/assets
